<a href="https://colab.research.google.com/github/estebanvz/redes_neuronales_pytorch/blob/aula-1/Aula1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alura - Redes Neuronales: Deep Learning con PyTorch

In [ ]:
TAMANO = 10000000

In [ ]:
import numpy as np
def funcion_activacion(x):
  if(x>=0.0):
    return 1
  else:
    return 0
class Perceptron():
  def __init__(self, datos_entrada):
    self.datos_entrada = np.array(datos_entrada)
    self.pesos = np.array([1.0,-1.0,1.0]*TAMANO)
  def forward(self,):
    self.datos_salida = self.datos_entrada @ self.pesos
    return funcion_activacion(self.datos_salida)


In [ ]:
%%time
datos_entrada = [0.22,0.170,0.80]*TAMANO
perceptron = Perceptron(datos_entrada)
prediccion = perceptron.forward()
prediccion

CPU times: user 3.57 s, sys: 18.1 ms, total: 3.59 s
Wall time: 3.54 s


1

In [ ]:
import torch
gpu = torch.device("cuda")
class PerceptronTorch():
  def __init__(self, datos_entrada):
    self.datos_entrada = torch.tensor(datos_entrada).to(gpu)
    self.pesos = torch.tensor([1.0,-1.0,1.0]*TAMANO).to(gpu)
  def forward(self,):
    self.datos_salida = self.datos_entrada @ self.pesos
    return funcion_activacion(self.datos_salida)

In [ ]:
%%time
datos_entrada = [0.22,0.170,0.80]*TAMANO
perceptron = PerceptronTorch(datos_entrada)
prediccion = perceptron.forward()
prediccion

CPU times: user 2.59 s, sys: 65.6 ms, total: 2.65 s
Wall time: 2.61 s


1